In [1]:
import copy
import pandas as pd
from sklearn import datasets, model_selection
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

<h3>Load Datasets</h3>

In [2]:
cancer = datasets.load_breast_cancer()
dtcancer = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
dttennis = pd.read_csv("../data/play_tennis.csv")

<h3>Split Datasets</h3>

In [3]:
ctrain, ctest = model_selection.train_test_split(dtcancer, test_size=0.2, train_size=0.8, random_state=1)
_tntrain, _tntest = model_selection.train_test_split(dttennis, test_size=0.2, train_size=0.8, random_state=1)

<h3>Encode Play Tennis Column</h3>

In [4]:
label_encoder = LabelEncoder()
tntrain = copy.deepcopy(_tntrain)
tntest = copy.deepcopy(_tntest)
for cl in tntrain:
    tntrain[cl] = label_encoder.fit_transform(tntrain[cl])
    tntest[cl] = label_encoder.fit_transform(tntest[cl])

<h3>Decision Tree Classifier</h3>

<h3>Id3 Estimator</h3>

<h3>KMeans</h3>

In [5]:
ckmeans = KMeans(n_clusters=2, random_state=42).fit(ctrain)
ckmeanspredict = ckmeans.predict(ctest)

tnkmeans = KMeans(n_clusters=2, random_state=42).fit(tntrain)
tnkmeanspredict = tnkmeans.predict(tntest)

# accuracy_score(ckmeans.labels_, ckmeanspredict, normalize=False)
# f1_score(ckmeans.labels_, ckmeanspredict)

<h3>Logistic Regression</h3>

<h3>Neural Network</h3>

In [6]:
# to do tomorrow


<h3>SVM</h3>

In [7]:
# to do tomorrow
